# Parallel computing with Dask

## Authors & Contributors
### Authors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)
- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)

### Contributors
- Anne Fouilloux, University of Oslo (Norway), [@annefou](https://github.com/annefou)

<div class="alert alert-info">
<i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> Overview
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>What is Dask?</li>
        <li>How can I parallelize my data analysis with Dask?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn about Dask</li>
        <li>Learn about Dask Gateway, Dask Client, Scheduler, Workers</li>
        <li>Understand out-of-core and speed-up limitations</li>
    </ul>
</div>

## Context


We will be using [Dask](https://docs.dask.org/) with [Xarray](https://docs.xarray.dev/en/stable/) to parallelize our data analysis. The analysis is very similar to what we have done in previous episodes but this time we will use data on a global coverage that we read from a shared catalog (stored online in the Pangeo EOSC Openstack Object Storage).

### Data

In this episode, we will be using Global Long Term Statistics (1999-2019) product provided by the [Copernicus Global Land Service over Lombardia](https://land.copernicus.eu/global/index.html) and access them through [S3-comptabile storage](https://en.wikipedia.org/wiki/Amazon_S3) ([OpenStack Object Storage "Swift"](https://wiki.openstack.org/wiki/Swift)) with a data catalog we have created and made publicly available.

## Setup

This episode uses the following Python packages:

- pooch {cite:ps}`e-pooch-Uieda2020`
- s3fs {cite:ps}`e-s3fs-2016`
- xarray {cite:ps}`e-xarray-hoyer2017` with [`netCDF4`](https://pypi.org/project/h5netcdf/) and [`h5netcdf`](https://pypi.org/project/h5netcdf/) engines
- hvplot {cite:ps}`e-holoviews-rudiger2020`
- dask {cite:ps}`e-dask-2016`
- graphviz {cite:ps}`e-graphviz-Ellson2003`
- numpy {cite:ps}`e-numpy-harris2020`
- pandas {cite:ps}`e-pandas-reback2020`
- geopandas {cite:ps}`e-geopandas-jordahl2020`

Please install these packages if not already available in your Python environment (you might want to take a look at [the Setup page of the tutorial](https://pangeo-data.github.io/foss4g-2022/before/setup.html)).
### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

## Parallelize with Dask

We know from previous chapter [chunking_introduction](./chunking_introduction.ipynb) that chunking is key for analyzing large datasets. In this episode, we will learn to parallelize our data analysis using [Dask](https://docs.dask.org/) on our chunked dataset. 

### What is [Dask](https://docs.dask.org/) ?

**Dask** scales the existing Python ecosystem: with very or no changes in your code, you can speed-up computation using Dask or process bigger than memory datasets.

- Dask is a flexible library for parallel computing in Python.
- It is widely used for handling large and complex Earth Science datasets and speed up science.
- Dask is powerful, scalable and flexible. It is the leading platform today for data analytics at scale.
- It scales natively to clusters, cloud, HPC and bridges prototyping up to production.
- The strength of Dask is that is scales and accelerates the existing Python ecosystem e.g. Numpy, Pandas and Scikit-learn with few effort from end-users.

It is interesting to note that at first, [Dask has been created to handle data that is larger than memory, on a single computer](https://coiled.io/blog/history-dask/). It then was extended with Distributed to compute data in parallel over clusters of computers.

#### How does Dask scale and accelerate your data analysis?

[Dask proposes different abstractions to distribute your computation](https://docs.dask.org/en/stable/10-minutes-to-dask.html). In this _Dask Introduction_ section, we will focus on [Dask Array](https://docs.dask.org/en/stable/array.html) which is widely used in pangeo ecosystem as a back end of Xarray.

As shown in the [previous section](./chunking_introduction.ipynb) Dask Array is based on chunks.
Chunks of a Dask Array are well-known Numpy arrays. By transforming our big datasets to Dask Array, making use of chunk, a large array is handled as many smaller Numpy ones and we can compute each of these chunks independently.

![Dask and Numpy](https://examples.dask.org/_images/dask-array-black-text.svg)



<div class="alert alert-info">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Warning</b>
    <br>
    <ul>
        <li>`Xarray` uses Dask Arrays instead of Numpy when chunking is enabled, and thus all Xarray operations are performed through Dask, which enables distributed processing. </li>
    </ul>
</div>


#### How does Xarray with Dask distribute data analysis?

When we use chunks with `Xarray`, the real computation is only done when needed or asked for, usually when invoking `compute()` function. Dask generates a **task graph** describing the computations to be done. When using [Dask Distributed](https://distributed.dask.org/en/stable/) a **Scheduler** distributes these tasks across several **Workers**.

![Xarray with dask](../figures/dask-xarray-explained.png)

:::{tip}
A Dask client can also be created on a single machine (for instance your laptop) e.g. there is no need to have dedicated computational resources. However, speedup will only be limited to your single machine resources if you do not have dedicated computational resources!
:::

#### What is a Dask Cluster?

A Dask Distributed cluster is made of two main components:

- a Scheduler, responsible for handling computations graph and distributing tasks to Workers.
- One or several (up to 1000s) Workers, computing individual tasks and storing results and data into distributed memory.

To interact with a Dask Cluster, a Client object must also be used.

![Dask Distributed Cluster](https://user-images.githubusercontent.com/306380/66413985-27111600-e9be-11e9-9995-8f418ff48f8a.png)

Dask clusters can be deployed in many backend (Locally or on distributed infrastructures like Kubernetes, YaRN or HPC centers). In these cases, a Cluster object in responsible of deploying and scaling a Dask Cluster on the underlying resources.

![Dask Cluster deployment](https://docs.dask.org/en/stable/_images/dask-cluster-manager.svg)

## Using Dask Distributed on a single server

There are different methods to use Dask depending on the underlying infrastructure. For this workshop, according to the Pangeo EOSC deployment, you will learn how to use Dask Gateway to manage Dask clusters over Kubernetes, allowing to run our data analysis in parallel e.g. distribute tasks across several workers.

However, you do not always need to access a multi-node Dask cluster. It is very handy to prototype or even run data analysis on your own laptop, or a small server. So let's keep it simple for now and learn how to create a local Dask cluster to distribute some work.

### Create a local Dask cluster
 
The Dask `Client` is what allows you to interact with Dask Clusters. When using Dask Distributed, you always need to create a `Client` object. Once a `Client` has been created, it will be used by default by each call to a Dask API, even if you do not explicitly use it.

No matter the Dask API (e.g. Arrays, Dataframes, Delayed, Futures, etc.) that you use, under the hood, Dask will create a Directed Acyclic Graph (DAG) of tasks by analysing the code. Client will be responsible to submit this DAG to the Scheduler along with the final result you want to compute. It will also gather results from the Workers, and aggregates it back in its underlying Python process.

Using `Client()` function with no argument, you will create a local Dask cluster with a number of workers and threads per worker corresponding to the number of cores in the local machine. Here, we are running this notebook in the cloud, so the number of cores is the number of cores on the cloud computing resources you've been given (not on your laptop).

In [ ]:
from distributed import Client

client = Client()   # create a local dask cluster on the local machine.
client

Inspecting the `Cluster Info` section above gives us information about the created cluster: we have 2 or 4 workers and the same number of threads (e.g. 1 thread per worker). 

You can also create a local cluster with the `LocalCluster` constructor and use `n_workers` and `threads_per_worker` to anually specify the number of processes and threads you want to use. For instance, we could use `n_workers=2` and `threads_per_worker=2` (the total number of threads would be 4 and each Worker would have 2 threads). This is sometimes preferable (in terms of performance) but out of scope of this tutorial.

<div class="alert alert-info">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Warning</b>
    <br>
    <ul>
        <li>Dask will try to hold data on the memory, then try to spill that to hard disk of worker.  If you would like to avoid that dask worker use your local disk ( it will slow down your computation), you can use following command.
        <li>import dask.distributed</li>
        <li>dask.config.set({"distributed.worker.memory.spill": 0}) </li>
    </ul>
</div>

### Dask Dashboard

Dask comes with a really handy interface: the Dask Dashboard.

The Dashboard link above will not work, but you can open it using the following link:

https://pangeo-foss4g.vm.fedcloud.eu/jupyterhub/user/_user_/proxy/8787/status

It's really helpfull to understand your computation and how it is distributed.

### A little Dask computation

Launch the below cell which tries to approximate Pi using Dask Arrays, and watch what is going on on the Dashboard.


In [ ]:
import dask.array as da

sample = 1_000_000_000  # <- this is huge!
xxyy = da.random.uniform(-1, 1, size=(2, sample))
norm = da.linalg.norm(xxyy, axis=0)
summ = da.sum(norm <= 1)
insiders = summ.compute()
pi = 4 * insiders / sample
print("pi ~= {}".format(pi))

## Local Dask Distributed computations on our dataset

Lets open dataset from catalogue we made before, select a single location and visualize the task graph generated by Dask.


In [ ]:
import xarray as xr

catalogue="https://object-store.cloud.muni.cz/swift/v1/foss4g-catalogue/c_gls_NDVI-LTS_1999-2019.json"
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo":catalogue
                    },
        "consolidated": False
    }
)
LTS

In [ ]:
save=LTS.sel(lat=45.50, lon=9.36, method='nearest')['min'].mean()
save.data

Did you notice something on the Dask Dashboard when running the two previous cells?

It is because currently, we didn't execute anything, we just build a Dask task graph, but did not ask Dask to return a result.

As you look at 'task count' we see 6121 task for just for computing a mean on 36 temporal steps.

To avoid unecessary operations, we optimize the task graph using `optimize`, and verify the graph.

### Optimize the task graph

In [ ]:
import dask

In [ ]:
(save,) = dask.optimize(save)
save.data

Now our task is reduced to 121. Lets try to visualise it:

In [ ]:
save.data.visualize()

### Compute on the dask workers

In [ ]:
save.compute()

Calling compute on our Xarray object triggered the execution on Dask Cluster side. 

You should be able to see some tasks executing by looking at the Dashboard. Once it is finished, the result of the computation is returned.

### Close client to terminate local dask cluster

The `Client` and associated `LocalCluster` object will be automatically closed when your Python session ends. When using Jupyter notebooks, we recommend to close it explicitely whenever you are done with your local Dask cluster.

In [ ]:
client.close()

## Set up Dask Gateway

When we want to scale out our data analysis and cannot only use on machine, we need to be able to access a multi-node Dask cluster. 

On the EOSC Pangeo infrastructure, we can use Dask Gateway to manage Dask clusters and run our data analysis in parallel e.g. distribute tasks across several workers.

As Dask Gateway is configured by default on this ifnrastructure, you just need to execute the following cells.

In [ ]:
from dask_gateway import Gateway
gateway = Gateway()

### Create a new Dask cluster with the Dask Gateway

In [ ]:
cluster = gateway.new_cluster()
cluster.scale(4)
cluster

### Open the Dask Dashboard by clicking on the link above!!

### Get a client from the Dask Gateway Cluster

As stated above, creating a Dask `Client` is mandatory in order to perform following Daks computations on your Dask Cluster.

In [ ]:
## Please don't execute this cell, it is needed for building the Jupyter Book
cluster = None

In [ ]:
from distributed import Client

if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

## Global LTS computation

In the previous episode, we used Long-term Timeseries for the region of Lombardy e.g. a very small area that was extracted upfront for simplicity. Now we will use the original dataset that has a global coverage, and work directly on it to extract our AOI and perform computations.

### Read from online kerchunked consolidated dataset

 We will access Long Term TimeSeries of NDVI statistics from OpenStack Object Storage using the Zarr metadata generated with kerchunk, prepared in [previous chunking_introduction](./chunking_introduction.ipynb) section.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
catalogue="https://object-store.cloud.muni.cz/swift/v1/foss4g-catalogue/c_gls_NDVI-LTS_1999-2019.json"
LTS = xr.open_mfdataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo":catalogue
                    },
        "consolidated": False
    }
)
LTS

By inspecting any of the variable on the representation above, you'll see that each data array represent __about 85GiB of data__, so much more thant what is available on this notebook server, and even on the Dask Cluster we created above. But thants to chunking, we can still analyze it!

### Visualize LTS statistics

Don't forget to have a look at the Dashboard when executing the cells below.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(1, figsize=[15,5])

# Fix extent
minval = 0.0
maxval = 0.9

itime=0 # plot the first date

# Plot 1 for min subplot argument (nrows, ncols, nplot)
# here 1 row, 2 columns and 1st plot
ax1 = plt.subplot(1, 2, 1)
LTS.isel(time=itime, lat=slice(1000,2000), lon=slice(1000,2000))['min'].plot(ax=ax1)
# Plot 2 for max
# 2nd plot
ax2 = plt.subplot(1, 2, 2)
LTS.isel(time=itime, lat=slice(1000,2000), lon=slice(1000,2000))['max'].plot(ax=ax2)

# Title for both plots
fig.suptitle('LTS NDVI statistics (Minimum and Maximum)', fontsize=20)

### Fix time coordinate

As observed data are coming with a predefined year. To let xarray automatically align the LTS with the lastest NDVI values, the time dimension needs to be shifted to the NDVI values.

In [ ]:
dates_2022 = pd.date_range('20220101', '20221231')
time_list = dates_2022[np.isin(dates_2022.day, [1,11,21])]

In [ ]:
LTS = LTS.assign_coords(time=time_list)
LTS

### Clip LTS over Lombardia
As in previous episodes, we use a shapefile over Italy to select data over this Area of Interest (AOI).

In [ ]:
import geopandas as gpd

In [ ]:
try:
    GAUL = gpd.read_file('Italy.geojson')
except:
    GAUL = gpd.read_file('zip+https://mars.jrc.ec.europa.eu/asap/files/gaul1_asap.zip') 

In [ ]:
AOI_name = 'Lombardia'
AOI = GAUL[GAUL.name1 == AOI_name]
AOI_poly = AOI.geometry
AOI_poly

We first select a geographical area that covers Lombardia (so that we have a first reduction from the global coverage) and then clip using the shapefile to avoid useless pixels.

In [ ]:
LTS = LTS.sel(lat=slice(46.5,44.5), lon=slice(8.5,11.5))
LTS.rio.write_crs(4326, inplace=True)

In [ ]:
LTS = LTS.rio.clip(AOI_poly, crs=4326)

In [ ]:
LTS

In [ ]:
%%time
LTS.compute()

In [ ]:
LTS

In [ ]:
%%time
LTS_min = LTS['min']
(LTS_min,)=dask.optimize(LTS_min)
LTS_min.data.visualize()

In [ ]:
%%time
LTS_min.compute()

In [ ]:
%%time
LTS_max = LTS['max']
(LTS_max,)=dask.optimize(LTS_max)
LTS_max.data.visualize()

In [ ]:
%%time
LTS_max.compute()

## Get NDVI for 2022 over Lombardia

We re-use the file we created during the first episode. If the file is missing it will be downloaded from Zenodo.

In [ ]:
import pooch
try:
    cgls_ds = xr.open_dataset('C_GLS_NDVI_20220101_20220701_Lombardia_S3_2_masked.nc')
except:
    cgls_file = pooch.retrieve(
        url="https://zenodo.org/record/6969999/files/C_GLS_NDVI_20220101_20220701_Lombardia_S3_2_masked.nc",
        known_hash="md5:be3f16913ebbdb4e7af227f971007b22",
        path=f".",)    
    cgls_ds = xr.open_dataset(cgls_file)

In [ ]:
NDVI_AOI = cgls_ds.NDVI.rio.write_crs(4326, inplace=True)

In [ ]:
NDVI_AOI = NDVI_AOI.rio.clip(AOI_poly, crs=4326)

In [ ]:
NDVI_AOI

The nominal spatial resolution of the Long term statistics is 1km. As the current NDVI product has a nominal spatial resolution of 300m a re projection is needed. RioXarray through RasterIO that wraps the GDAL method can take care of this. More info about all the options can be found [here](https://rasterio.readthedocs.io/en/stable/api/rasterio.warp.html#rasterio.warp.reproject).

In [ ]:
NDVI_1k = NDVI_AOI.rio.reproject_match(LTS)

In [ ]:
NDVI_1k = NDVI_1k.rename({'x': 'lon', 'y':'lat'})

In [ ]:
VCI = ((NDVI_1k - LTS['min']) / (LTS['max'] - LTS['min'])) * 100

In [ ]:
VCI

In [ ]:
VCI.name = 'VCI'

In [ ]:
VCI

In [ ]:
%%time
VCI_c = VCI.compute()

In [ ]:
from hvplot import xarray
VCI.hvplot(x = 'lat', y = 'lon',
           cmap='RdYlGn', clim=(-200,+200), alpha=0.7,
           geo=True, tiles= 'CartoLight',
           title=f'CGLS VCI {AOI_name} {VCI.isel(time=-1).time.dt.date.data}',
           width=800, height=700,
           )

Now you have catalogue, original data source, both on cloud space, thus even from dask workers which do not have access to your NFS local disk space, data are accessible.
Now you are ready to parallelize your analysis using dask workers from dask gateway!

In [ ]:
client.close()

In [ ]:
cluster.shutdown()

## Packages citation

```{bibliography}
:style: alpha
:filter: topic % "dask" and topic % "package"
:keyprefix: e-
```